# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [2]:
! pip install -q schedule pytest # установка библиотек, если ещё не


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [4]:
from dataclasses import dataclass
from typing import Dict, Optional


@dataclass
class BookData:
    """
    Data class to hold book information scraped from Books to Scrape website.
    
    Attributes:
        title: Book title
        price: Book price as string (e.g., "£51.77")
        rating: Star rating (e.g., "Three", "Four", "Five")
        availability: Stock availability information
        description: Book description text
        product_info: Dictionary containing product information table data
    """
    title: str
    price: str
    rating: str
    availability: str
    description: str
    product_info: Dict[str, str]


def _extract_text(soup: BeautifulSoup, tag: str, default: str, **kwargs) -> str:
    """Universal text extraction from HTML element."""
    element = soup.find(tag, **kwargs)
    return element.get_text(strip=True) if element else default


def _extract_attribute(soup: BeautifulSoup, tag: str, attr_name: str, default: str, **kwargs) -> str:
    """Universal attribute extraction from HTML element."""
    element = soup.find(tag, **kwargs)
    if element and attr_name in element.attrs:
        attr_value = element.attrs[attr_name]
        if isinstance(attr_value, list) and len(attr_value) > 1:
            return attr_value[1]  # For rating: get second class name
        return str(attr_value)
    return default


def _extract_text_from_next_sibling(soup: BeautifulSoup, parent_tag: str, 
                                   sibling_tag: str, default: str, **kwargs) -> str:
    """Extract text from next sibling element."""
    parent = soup.find(parent_tag, **kwargs)
    if parent:
        sibling = parent.find_next_sibling(sibling_tag)
        if sibling:
            return sibling.get_text(strip=True)
    return default


def _extract_table_data(soup: BeautifulSoup, tag: str, **kwargs) -> Dict[str, str]:
    """Extract key-value pairs from HTML table."""
    data = {}
    table = soup.find(tag, **kwargs)
    if table:
        rows = table.find_all("tr")
        for row in rows:
            th = row.find("th")
            td = row.find("td")
            if th and td:
                key = th.get_text(strip=True)
                value = td.get_text(strip=True)
                data[key] = value
    return data


def get_book_data(book_url: str) -> Optional[BookData]:
    """
    Scrape book information from a single book page on Books to Scrape website.
    
    Args:
        book_url: URL of the book page to scrape
        
    Returns:
        BookData object containing scraped book information, or None if scraping fails
    """
    try:
        response = requests.get(book_url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        return BookData(
            title=_extract_text(soup, "h1", "Unknown Title"),
            price=_extract_text(soup, "p", "Price not available", class_="price_color"),
            rating=_extract_attribute(soup, "p", "class", "Not rated", class_="star-rating"),
            availability=_extract_text(soup, "p", "Availability unknown", class_="instock availability"),
            description=_extract_text_from_next_sibling(soup, "div", "p", "No description available", id="product_description"),
            product_info=_extract_table_data(soup, "table", class_="table table-striped")
        )
        
    except requests.RequestException as e:
        print(f"Network error occurred: {e}")
        return None
    except AttributeError as e:
        print(f"Parsing error occurred: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return None

In [5]:
# Test the function with the provided URL
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
book_data = get_book_data(book_url)

if book_data:
    print(f"\nBook: {book_data.title}")
    print(f"Price: {book_data.price}")
    print(f"Rating: {book_data.rating}")
    print(f"Availability: {book_data.availability}")
    print("\nDescription:")
    print(f"{book_data.description[:200]}..." if len(book_data.description) > 200 else book_data.description)
    print("\nProduct Information:")
    for key, value in book_data.product_info.items():
        print(f"  {key}: {value}")
else:
    print("Failed to scrape book data")


Book: A Light in the Attic
Price: £51.77
Rating: Three
Availability: In stock (22 available)

Description:
It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverste...

Product Information:
  UPC: a897fe39b1053632
  Product Type: Books
  Price (excl. tax): £51.77
  Price (incl. tax): £51.77
  Tax: £0.00
  Availability: In stock (22 available)
  Number of reviews: 0


## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [6]:
import json
from pathlib import Path
from typing import List
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import asdict


def _collect_book_urls() -> List[str]:
    """
    Collect all book URLs by iterating through catalog pages.
    
    Returns:
        List of absolute book URLs
    """
    book_urls = []
    page = 1
    base_url = "http://books.toscrape.com/catalogue"
    
    while True:
        # Construct catalog page URL - start with page-1.html
        page_url = f"{base_url}/page-{page}.html"
        
        try:
            # Fetch and parse catalog page
            response = requests.get(page_url, timeout=10)
            if response.status_code != 200:
                break
                
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Find all book links on this catalog page
            articles = soup.find_all("article", class_="product_pod")
            if not articles:
                break
                
            for article in articles:
                # Extract book URL from article
                link_element = article.find("h3").find("a")
                if link_element:
                    link = link_element["href"]
                    # Convert relative to absolute URL
                    absolute_url = f"http://books.toscrape.com/catalogue/{link}"
                    book_urls.append(absolute_url)
            
            print(f"Catalog page {page}: found {len(articles)} books")
            page += 1
            
        except requests.RequestException as e:
            print(f"Error fetching catalog page {page}: {e}")
            break
    
    return book_urls


def scrape_books(is_save: bool = False, max_workers: int = 10) -> List[BookData]:
    """
    Scrape all books from Books to Scrape catalog using concurrent execution.
    
    Args:
        is_save: If True, save results to books_data.txt
        max_workers: Number of concurrent workers for parallel scraping
        
    Returns:
        List of BookData objects for all scraped books
    """
    # Phase 1: Collect all book URLs from catalog pages
    print("Collecting book URLs from catalog pages...")
    book_urls = _collect_book_urls()
    print(f"Found {len(book_urls)} books to scrape")
    
    if not book_urls:
        print("No books found to scrape")
        return []
    
    # Phase 2: Scrape books concurrently
    print(f"Scraping books with {max_workers} workers...")
    books = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_url = {executor.submit(get_book_data, url): url for url in book_urls}
        
        completed = 0
        for future in as_completed(future_to_url):
            book_data = future.result()
            if book_data:
                books.append(book_data)
            completed += 1
            if completed % 100 == 0:
                print(f"Progress: {completed}/{len(book_urls)} books scraped")
    
    print(f"Successfully scraped {len(books)} books")
    
    # Save to file if requested
    if is_save:
        output_path = Path("books_data.txt")
        
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump([asdict(book) for book in books], f, indent=2, ensure_ascii=False)
        
        print(f"Saved to {output_path}")
    
    return books

In [7]:
# Test the scrape_books function with a small number of workers for testing
print("Testing scrape_books function...")
res = scrape_books(is_save=True, max_workers=5)
print(f"\nResult type: {type(res)}")
print(f"Number of books scraped: {len(res)}")

if res:
    print("\nFirst book example:")
    print(f"  Title: {res[0].title}")
    print(f"  Price: {res[0].price}")
    print(f"  Rating: {res[0].rating}")
else:
    print("No books were scraped")

Testing scrape_books function...
Catalog page 1: found 20 books
Catalog page 2: found 20 books
Catalog page 3: found 20 books
Catalog page 4: found 20 books
Catalog page 5: found 20 books
Catalog page 6: found 20 books
Catalog page 7: found 20 books
Catalog page 8: found 20 books
Catalog page 9: found 20 books
Catalog page 10: found 20 books
Catalog page 11: found 20 books
Catalog page 12: found 20 books
Catalog page 13: found 20 books
Catalog page 14: found 20 books
Catalog page 15: found 20 books
Catalog page 16: found 20 books
Catalog page 17: found 20 books
Catalog page 18: found 20 books
Catalog page 19: found 20 books
Catalog page 20: found 20 books
Catalog page 21: found 20 books
Catalog page 22: found 20 books
Catalog page 23: found 20 books
Catalog page 24: found 20 books
Catalog page 25: found 20 books
Catalog page 26: found 20 books
Catalog page 27: found 20 books
Catalog page 28: found 20 books
Catalog page 29: found 20 books
Catalog page 30: found 20 books
Catalog page 31:

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [8]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

def run_scheduler():
    """
    Set up and run the automated daily data collection scheduler.
    
    The scheduler runs the book scraping function every day at 19:00 (7 PM)
    and saves the results to books_data.txt. The function runs in an infinite
    loop with 60-second intervals to check for scheduled tasks.
    
    The scheduler can be stopped with Ctrl+C (KeyboardInterrupt).
    """
    from datetime import datetime
    
    def scheduled_scraping():
        """Wrapper function for scheduled book scraping."""
        print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Starting scheduled book scraping...")
        
        try:
            # Run the scraping function with file saving enabled
            books = scrape_books(is_save=True, max_workers=5)
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Successfully scraped {len(books)} books")
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Data saved to books_data.txt")
            
        except Exception as e:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Error during scheduled scraping: {e}")
    
    # Schedule the scraping function to run daily at 19:00
    schedule.every().day.at("19:00").do(scheduled_scraping)
    
    print("Scheduler started! Book scraping is scheduled for 19:00 daily.")
    print("Press Ctrl+C to stop the scheduler.")
    
    try:
        # Infinite loop to check for scheduled tasks
        while True:
            schedule.run_pending()
            time.sleep(60)  # Check every minute to avoid overloading the system
            
    except KeyboardInterrupt:
        print("\nScheduler stopped by user.")
    except Exception as e:
        print(f"Scheduler error: {e}")


def test_scheduler_with_custom_time(test_time: str = "14:30"):
    """
    Test the scheduler functionality with a custom time for immediate verification.
    
    Args:
        test_time: Time in HH:MM format to schedule the test run (default: "14:30")
    """
    from datetime import datetime
    
    def test_scraping():
        """Test function for scheduler verification."""
        print(f"\n[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] TEST: Starting scheduled scraping at {test_time}...")
        
        try:
            # Run a small test with just a few books
            print("Running test with limited scope...")
            books = scrape_books(is_save=False, max_workers=2)
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] TEST: Successfully scraped {len(books)} books")
            
        except Exception as e:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] TEST: Error during test scraping: {e}")
    
    # Clear any existing schedules
    schedule.clear()
    
    # Schedule the test function at the specified time
    schedule.every().day.at(test_time).do(test_scraping)
    
    current_time = datetime.now().strftime('%H:%M')
    print(f"Test scheduler started! Test scraping is scheduled for {test_time} (current time: {current_time})")
    print("Press Ctrl+C to stop the test scheduler.")
    
    try:
        # Run for a limited time to test the functionality
        start_time = time.time()
        while time.time() - start_time < 300:  # Run for 5 minutes max
            schedule.run_pending()
            time.sleep(10)  # Check every 10 seconds for testing
            
        print("Test scheduler completed (5-minute timeout)")
        
    except KeyboardInterrupt:
        print("\nTest scheduler stopped by user.")
    except Exception as e:
        print(f"Test scheduler error: {e}")


# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [9]:
from datetime import datetime, timedelta
print("Testing scheduler with custom time (next minute)...")

next_minute = (datetime.now() + timedelta(minutes=1)).strftime('%H:%M')
test_scheduler_with_custom_time(next_minute)

Testing scheduler with custom time (next minute)...
Scheduling test for 21:32
Test scheduler started! Test scraping is scheduled for 21:32 (current time: 21:31)
Press Ctrl+C to stop the test scheduler.

[2025-10-28 21:32:04] TEST: Starting scheduled scraping at 21:32...
Running test with limited scope...
Catalog page 1: found 20 books
Catalog page 2: found 20 books
Catalog page 3: found 20 books
Catalog page 4: found 20 books
Catalog page 5: found 20 books
Catalog page 6: found 20 books
Catalog page 7: found 20 books
Catalog page 8: found 20 books
Catalog page 9: found 20 books
Catalog page 10: found 20 books
Catalog page 11: found 20 books
Catalog page 12: found 20 books
Catalog page 13: found 20 books
Catalog page 14: found 20 books
Catalog page 15: found 20 books
Catalog page 16: found 20 books
Catalog page 17: found 20 books
Catalog page 18: found 20 books
Catalog page 19: found 20 books
Catalog page 20: found 20 books
Catalog page 21: found 20 books
Catalog page 22: found 20 books

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [14]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest ../tests/test_scraper.py -v

============================= test session starts ==============================
platform darwin -- Python 3.12.4, pytest-8.4.2, pluggy-1.6.0 -- /Users/m/edu/mipt/mipt-hw-3/.venv/bin/python3
cachedir: .pytest_cache
rootdir: /Users/m/edu/mipt/mipt-hw-3
plugins: timeout-2.4.0
collected 8 items                                                              

../tests/test_scraper.py::TestGetBookData::test_get_book_data_returns_dict_with_required_keys PASSED [ 12%]
../tests/test_scraper.py::TestGetBookData::test_book_data_fields_are_correct PASSED [ 25%]
../tests/test_scraper.py::TestGetBookData::test_get_book_data_handles_network_error PASSED [ 37%]
../tests/test_scraper.py::TestScrapeBooks::test_scrape_books_returns_list PASSED [ 50%]
../tests/test_scraper.py::TestScrapeBooks::test_scrape_books_handles_empty_catalog PASSED [ 62%]
../tests/test_scraper.py::TestScrapeBooks::test_scrape_books_handles_failed_scraping PASSED [ 75%]
../tests/test_scraper.py::TestCollectBookUrls::test_collect_boo

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```